In [3]:
import os
from Bio import SeqIO

In [4]:
!pwd

/Users/ben/Documents/work_related/Canberra/RJ_lab_projects/Rust_genome_sequencing/PST79/analysis/PacBio_SCH1743/FALCON_0_4_assemblies/p_assemblies/v9_1/scripts


In [5]:
#here is the in file defined containing alternative 
file_path ='/Users/ben/Documents/work_related/Canberra/RJ_lab_projects/Rust_genome_sequencing/PST79/analysis/PacBio_SCH1743/FALCON_0_4_assemblies/p_assemblies/v9_1/122016_assembly'
file_name='Pst_E104_v1_ph_ctg.fa'
genome_prefix = 'Pst_E104_v1'
bash_script_q= genome_prefix+"_ph_ctg_qmapping.sh"
bash_script_g=genome_prefix+"_ph_ctg_gmapping.sh"
bash_script_nucmer_assemlytics = genome_prefix +"_ncumer_assemblyticsmapping.sh"
outdir = '/Users/ben/Documents/work_related/Canberra/RJ_lab_projects/Rust_genome_sequencing/PST79/analysis/PacBio_SCH1743/FALCON_0_4_assemblies/p_assemblies/v9_1/Pst_E104_v1'

In [6]:
total_seq_recs = 0
p_contigs =[]
a_contigs = []
a_count =0
p_count =0
for seq_record in SeqIO.parse(file_path+'/'+file_name, "fasta"):
    if "hcontig" in seq_record.id:
        a_contigs.append(seq_record)
        a_count += 1
    else:
        p_contigs.append(seq_record)
        p_count += 1
    total_seq_recs += 1
total_seq_recs == p_count + a_count

True

In [7]:
os.chdir(outdir)

In [8]:
SeqIO.write(a_contigs,file_name[:-3]+'_h_ctg.fa' , 'fasta')
SeqIO.write(p_contigs,file_name[:-3]+'_p_ctg.fa' , 'fasta')

176

In [9]:
!ls

Pst_E104_v1_ph_ctg_h_ctg.fa nucmer_split
Pst_E104_v1_ph_ctg_p_ctg.fa


In [10]:
os.chdir('/Users/ben/Documents/work_related/Canberra/RJ_lab_projects/Rust_genome_sequencing/PST79/analysis/PacBio_SCH1743/FALCON_0_4_assemblies/p_assemblies/v9_1/Pst_E104_v1/nucmer_split')

In [14]:
!cp ../Pst_E104_v1_ph_ctg_h_ctg.fa .
!cp ../Pst_E104_v1_ph_ctg_p_ctg.fa .

In [21]:
outfq = open(bash_script_q, 'w')
outfq.write('#!/bin/bash\n')
outfg = open(bash_script_g, 'w')
outfg.write('#!/bin/bash\n')#parsing out P and corresponding A contigs and writing a short ncumer script that aligns them against each other
outfna = open(bash_script_nucmer_assemlytics, 'w')
outfna.write('#!/bin/bash\n')
for pseq_record in SeqIO.parse(file_name[:-3]+'_p_ctg.fa', 'fasta'):
    p_acontigs = []
    p_contig = pseq_record.id.split("_")[0]+"_"+pseq_record.id.split("_")[1]
    suffix = file_name[0:12]+p_contig+"_php"
    p_file = file_name[0:12]+"_"+p_contig+'.fa'
    SeqIO.write(pseq_record, p_file, 'fasta')
    for aseq_record in SeqIO.parse(file_name[:-3]+'_h_ctg.fa', 'fasta'):
        if aseq_record.id.split("_")[1]  == pseq_record.id.split("_")[1]:
            p_acontigs.append(aseq_record)
    a_file = file_name[0:12]+"_"+pseq_record.id.split("_")[0]+"_"+pseq_record.id.split("_")[1]+'_h_ctgs.fa'
    #if we have alternative contigs safe those too
    if p_acontigs != []:
        SeqIO.write(p_acontigs, a_file, 'fasta')
        outfq.write('nucmer '+p_file+' '+a_file+" > "+'out.delta\n')
        outfq.write('delta-filter -q '+'out.delta'+" > "+suffix+"_qfiltered.delta\n")
        outfq.write('show-coords -T '+suffix+"_qfiltered.delta > "+suffix+".qcoords\n")
        outfq.write('mummerplot -p '+suffix+'_qfiltered --png '+suffix+"_qfiltered.delta\n")
        outfq.write('mummerplot -c -p '+suffix+'_qfiltered_cov --png '+suffix+"_qfiltered.delta\n")
        #for g_file bash script
        outfg.write('nucmer '+p_file+' '+a_file+" > "+'out.delta\n')
        outfg.write('delta-filter -g '+'out.delta'+" > "+suffix+"_gfiltered.delta\n")
        outfg.write('show-coords -T '+suffix+"_gfiltered.delta > "+suffix+".gcoords\n")
        outfg.write('mummerplot -p '+suffix+'_gfiltered --png '+suffix+"_gfiltered.delta\n")
        outfg.write('mummerplot -c -p  '+suffix+'_gfiltered_cov --png '+suffix+"_gfiltered.delta\n")
        #for nucmer assemblytics out
        outfna.write('nucmer -maxmatch -l 100 -c 500 '+p_file+' '+a_file+' -prefix ' + suffix +'\n')
outfna.close()    
outfq.close()
outfg.close()  

In [20]:
!mkdir assemblytics
!cp * assemblytics

mkdir: assemblytics: File exists
cp: assemblytics is a directory (not copied).


In [45]:
outfnarun = open('Pst_E104_v1_assemblytics.sh', 'w')
outfnarun.write('#!/bin/bash\n')

files = !ls assemblytics/
delta_files = [x for x in files if 'delta' in x]
assemblytics_path = '/Users/ben/scripts/Assemblytics'
outfnarun.write('export PATH="%s":$PATH\n'% assemblytics_path)
outfnarun.write('#Assemblytics delta_file output_prefix unique_anchor_length path_to_R_scripts\n')
for delta in delta_files:
    delta_10kbp = delta[:-6] + '_10kbp'
    delta_50kbp = delta[:-6] + '_50kbp'
    outfnarun.write("mkdir %s\ncp %s %s\ncd %s\n" % (delta[:-6], delta, delta[:-6], delta[:-6]))
    outfnarun.write("Assemblytics %s %s 10000 %s\n" % (delta, delta_10kbp, assemblytics_path) )
    outfnarun.write("Assemblytics %s %s 50000 %s\ncd ..\n" % (delta, delta_50kbp, assemblytics_path) )

outfnarun.close()
!mv Pst_E104_v1_assemblytics.sh assemblytics/

In [39]:
delta_files[0][:-6]

'Pst_E104_v1_pcontig_000_php'

In [50]:
for pseq_record in SeqIO.parse('Pst_E104_v1__pcontig_000_h_ctgs.fa', 'fasta'):
    print(pseq_record.id +', %i'%len(pseq_record.seq))

hcontig_000_027, 41706
hcontig_000_029, 20699
hcontig_000_031, 23870
hcontig_000_024, 37946
hcontig_000_011, 10913
hcontig_000_005, 12961
hcontig_000_003, 2066231
hcontig_000_057, 36770
hcontig_000_050, 709509
hcontig_000_054, 294830
hcontig_000_036, 29234
hcontig_000_039, 26935
hcontig_000_049, 17471
hcontig_000_037, 27343
